# Adaptive Cruise Control

In [67]:
from cbf_opt import ControlAffineDynamics, ControlAffineCBF, ControlAffineASIF
import matplotlib.pyplot as plt
import hj_reachability as hj
import jax.numpy as jnp
import numpy as np
import seaborn as sns
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as anim

In [68]:
sys.path.insert(1, '../lib')
import refine_cbfs
import experiment_wrapper
from experiment_wrapper import RolloutTrajectory, TimeSeriesExperiment, StateSpaceExperiment

## Setup Problem (dynamics, environment and CBF)

Dynamics of the Adaptive Cruise Control problem (control affine), in the format of `cbf_opt`

`ACC_JNP_Dynamics` is required for usage with `hj_reachability` module

In [13]:
class ACCDynamics(ControlAffineDynamics):
    STATES = ['P', 'V', 'dP'] # position, velocity, distance between cars
    CONTROLS = ['ACC']
    def __init__(self, params, **kwargs):
        params['n_dims'] = 3
        params['control_dims'] = 1  # TODO: How to do this correctly?
        self.mass = params["mass"]
        self.g = params["g"]
        self.f0 = params["f0"]
        self.f1 = params["f1"]
        self.f2 = params["f2"]
        self.rolling_resistance = lambda x: self.f0 + self.f1 * x[..., 1] + self.f2 * x[..., 1] ** 2
        self.v0 = params["v0"]
        super().__init__(params, **kwargs)

    def state_jacobian(self, state, control, time=0.):
        J = np.repeat(np.zeros_like(state)[..., None], self.n_dims, axis=-1)
        J[..., 0, 1] = 1
        J[..., 1, 1] = -1 / self.mass * (self.f1 + 2 * self.f2 * state[..., 1])
        J[..., 2, 1] = -1 
        return J
        
    def open_loop_dynamics(self, state, time=0.):
        f = np.zeros_like(state)
        f[..., 0] = state[..., 1]
        f[..., 1] = -1 / self.mass * self.rolling_resistance(state)
        f[..., 2] = self.v0 - state[..., 1]
        return f

    def control_matrix(self, state, time=0.):
        B = np.repeat(np.zeros_like(state)[..., None], self.control_dims, axis=-1)
        B[..., 1, 0] = 1 / self.mass 
        return B


class ACCJNPDynamics(ACCDynamics):
    def __init__(self, params, **kwargs):
        super().__init__(params, **kwargs)
        self.rolling_resistance = lambda x: self.f0 + self.f1 * x[1] + self.f2 * x[1] ** 2

    def open_loop_dynamics(self, state, time=0.0):
        return jnp.array([state[1], -1 / self.mass * self.rolling_resistance(state), self.v0 - state[1]])

    def control_matrix(self, state, time=0.0):
        return jnp.expand_dims(jnp.array([0, 1 / self.mass, 0]), axis=-1)

    def disturbance_jacobian(self, state, time=0.0):
        return jnp.expand_dims(jnp.zeros(3), axis=-1)

Setting up the Dynamics with specific parameters for dry road and ice road

In [16]:
# common parameters
params_common = dict()
params_common["dt"] = 0.01
params_common["g"] = 9.81
params_common['v0'] = 14  # lead vehicle velocity
params_common['mass'] = 1650  # vehicle mass

In [17]:
# dry road
params_dry = params_common.copy()
params_dry['f0'] = 0.1 * 3  # friction coefficient
params_dry['f1'] = 5 * 3  # friction coefficient
params_dry['f2'] = 0.25 * 3  # friction coefficient
acc_dry = ACCDynamics(params_dry)
acc_jnp_dry = ACCJNPDynamics(params_dry)

# ice road
params_ice = params_common.copy()
params_ice['f0'] = 0.1
params_ice['f1'] = 5
params_ice['f2'] = 0.25
acc_ice = ACCDynamics(params_ice)
acc_jnp_ice = ACCJNPDynamics(params_ice)

# Initial Local CBFs

In [20]:
class ACCCBF(ControlAffineCBF):
    def __init__(self, dynamics, params, cd_limit, **kwargs):
        self.Th = params["Th"]
        self.cd = cd_limit
        super().__init__(dynamics, params, **kwargs)

    def vf(self, state, time=None):
        return state[..., 2] - self.Th * state[..., 1] - (state[..., 1] - self.dynamics.v0) ** 2 / (2 * self.cd * self.dynamics.g)

    def vf_dt_partial(self, state, time=None):
        return 0.0
    
    def _grad_vf(self, state, time=None):
        dvf_dx = np.zeros_like(state)
        dvf_dx[..., 1] = - self.Th  - (state[..., 1] - self.dynamics.v0) / (self.cd * self.dynamics.g)
        dvf_dx[..., 2] = 1.0
        return dvf_dx

Setting up the CBF with specific parameter instantiation

In [21]:
cbf_params = dict()
cbf_params["Th"] = 1.8
control_limit_dry = 0.3
control_limit_ice = 0.1

acc_cbf_dry = ACCCBF(acc_dry, cbf_params, control_limit_dry)
acc_jnp_cbf_dry = ACCCBF(acc_jnp_dry, cbf_params, control_limit_dry)

umax_dry = np.array([control_limit_dry * params_dry['mass'] * params_dry['g']])
umin_dry = - umax_dry

acc_cbf_ice = ACCCBF(acc_ice, cbf_params, control_limit_ice)
acc_jnp_cbf_ice = ACCCBF(acc_jnp_ice, cbf_params, control_limit_ice)

umax_ice = np.array([control_limit_ice * params_ice['mass'] * params_ice['g']])
umin_ice = - umax_ice

### Environment

In [22]:
dyn_reachability_jnp_dry = refine_cbfs.HJControlAffineDynamics(acc_jnp_dry, control_space=hj.sets.Box(jnp.array(umin_dry), jnp.array(umax_dry)))
dyn_reachability_dry = refine_cbfs.HJControlAffineDynamics(acc_dry, control_space=hj.sets.Box(jnp.array(umin_dry), jnp.array(umax_dry)))

dyn_reachability_jnp_ice = refine_cbfs.HJControlAffineDynamics(acc_jnp_ice, control_space=hj.sets.Box(jnp.array(umin_ice), jnp.array(umax_ice)))
dyn_reachability_ice = refine_cbfs.HJControlAffineDynamics(acc_ice, control_space=hj.sets.Box(jnp.array(umin_ice), jnp.array(umax_ice)))

low_bound = jnp.array([0., 10., 0.])
high_bound = jnp.array([500, 40., 100.])
nbr_pts = (101, 101, 101)
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(low_bound, high_bound), nbr_pts)
grid_np = refine_cbfs.Grid.from_lattice_parameters_and_boundary_conditions(hj.sets.Box(low_bound, high_bound), nbr_pts)

In [24]:
obstacle_dry = grid.states[..., 2] - acc_cbf_dry.Th * grid.states[..., 1]
obstacle_ice = grid.states[..., 2] - acc_cbf_ice.Th * grid.states[..., 1]
assert obstacle_dry.all() == obstacle_ice.all()

## Obtain the initial local CBFs using Dynamic Programming

In [26]:
acc_tabular_cbf_dry = refine_cbfs.TabularControlAffineCBF(acc_dry, dict(), grid=grid)
acc_jnp_tabular_cbf_dry = refine_cbfs.TabularControlAffineCBF(acc_jnp_dry, dict(), grid=grid)
acc_tabular_cbf_dry.tabularize_cbf(acc_cbf_dry)
acc_jnp_tabular_cbf_dry.tabularize_cbf(acc_jnp_cbf_dry)

acc_tabular_cbf_ice = refine_cbfs.TabularControlAffineCBF(acc_ice, dict(), grid=grid)
acc_jnp_tabular_cbf_ice = refine_cbfs.TabularControlAffineCBF(acc_jnp_ice, dict(), grid=grid)
acc_tabular_cbf_ice.tabularize_cbf(acc_cbf_ice)
acc_jnp_tabular_cbf_ice.tabularize_cbf(acc_jnp_cbf_ice)

In [31]:
# some setups
time = 0.
target_time = - 20.0
times = jnp.linspace(time, target_time, 101)
backwards_reachable_tube = lambda obstacle: (lambda t, x: jnp.minimum(x, obstacle))

In [30]:
# compute ice CBF values, it takes around 2 mins on desktop

solver_settings = hj.SolverSettings.with_accuracy("medium",
                                                  value_postprocessor=backwards_reachable_tube(obstacle_ice))
init_value = acc_jnp_tabular_cbf_ice.vf_table

# unsafe set: the last value is negative
target_values_hjr_ice = hj.solve(solver_settings, dyn_reachability_jnp_ice, grid, times, obstacle_ice)
target_values_ice = hj.solve(solver_settings, dyn_reachability_jnp_ice, grid, times, init_value)

100%|##########| 20.0000/20.0 [01:08<00:00,  3.45s/sim_s]
100%|##########| 20.0000/20.0 [01:07<00:00,  3.40s/sim_s]


In [32]:
# compute dry CBF values, it takes around 2 mins on desktop

solver_settings = hj.SolverSettings.with_accuracy("medium",
                                                  value_postprocessor=backwards_reachable_tube(obstacle_dry))
init_value_dry = acc_jnp_tabular_cbf_dry.vf_table
target_values_hjr_dry= hj.solve(solver_settings, dyn_reachability_jnp_dry, grid, times, obstacle_dry)
target_values_dry = hj.solve(solver_settings, dyn_reachability_jnp_dry, grid, times, init_value_dry)

100%|##########| 20.0000/20.0 [01:20<00:00,  4.01s/sim_s]
100%|##########| 20.0000/20.0 [01:19<00:00,  3.97s/sim_s]


In [33]:
# save data

dataset_path = "../dataset/acc_data"
if not os.path.isdir(dataset_path):
    os.makedirs(dataset_path)

with open(dataset_path + "/target_values_ice.npy", "wb") as f:
    jnp.save(f, target_values_ice)
with open(dataset_path + "/target_values_hjr_ice.npy", "wb") as f:
    jnp.save(f, target_values_hjr_ice)
with open(dataset_path + "/target_values_dry.npy", "wb") as f:
    jnp.save(f, target_values_dry)
with open(dataset_path + "/target_values_hjr_dry.npy", "wb") as f:
    jnp.save(f, target_values_hjr_dry)

In [34]:
# if you have computed values before, then just read them

target_values_ice = jnp.load(dataset_path + "/target_values_ice.npy")
target_values_hjr_ice = jnp.load(dataset_path + "/target_values_hjr_ice.npy")
target_values_dry = jnp.load(dataset_path + "/target_values_dry.npy")
target_values_hjr_dry = jnp.load(dataset_path + "/target_values_hjr_dry.npy")

In [36]:
assert jnp.sum(jnp.where(target_values_ice[-1] > 0, 1, 0) ) == jnp.sum(jnp.where(target_values_dry[-1] > 0, 1, 0) * jnp.where(target_values_ice[-1] > 0, 1, 0))

In [37]:
switch_position = 100
switch_radius = 1.
switch = abs(grid.states[..., 0] - switch_position) - switch_radius # considered as switching area

# the intersection of safe dry and unsafe ice regions
unsafe_ice_values = target_values_dry * target_values_ice

# safe switching area
safe_switch = jnp.maximum(switch, unsafe_ice_values[-1])

We need to refine dry CBF to make it aware of switching.
Now, turning the converged dry value function into a CBVF that interfaces with `cbf_opt`

In [39]:
# initialization with original local dry CBF

refined_dry_cbf = refine_cbfs.TabularControlAffineCBF(acc_dry, grid=grid_np)
refined_dry_cbf_jnp = refine_cbfs.TabularControlAffineCBF(acc_jnp_dry, grid=grid)
refined_dry_cbf.vf_table = np.array(target_values_dry[-1])
refined_dry_cbf_jnp.vf_table = target_values_dry[-1]

In [40]:
# compute refined dry CBF values, it takes around 1 min on desktop

new_obstacle = jnp.minimum(safe_switch, target_values_dry[-1])
solver_settings = hj.SolverSettings.with_accuracy("medium",
                                                  value_postprocessor=backwards_reachable_tube(new_obstacle))
init_value_refine_dry = refined_dry_cbf.vf_table
target_values_refine_dry = hj.solve(solver_settings, dyn_reachability_jnp_dry, grid, times, init_value_refine_dry)

100%|##########| 20.0000/20.0 [01:21<00:00,  4.06s/sim_s]


In [41]:
# save data
with open(dataset_path + "/target_values_refined_dry.npy", "wb") as f:
    jnp.save(f, target_values_refine_dry)

In [42]:
# read data
target_values_refine_dry = jnp.load(dataset_path + "/target_values_refined_dry.npy")

In [51]:
refined_cbf_dry = refine_cbfs.TabularControlAffineCBF(acc_dry, grid=grid_np)
refined_cbf_dry.vf_table = np.array(target_values_refine_dry[-1])

## Running an experiment

In [58]:
x0 = np.array([0, 30, 90])
n_sims_per_start = 1
t_sim = 20

Setting up the nominal controller
We simply tune a proporitional controller for the nominal controller

In [50]:
desired_vel = 35
feedback_gain = 200
nominal_policy = lambda x, t: np.atleast_1d(np.clip(-feedback_gain * (x[..., 1] - desired_vel), umin_dry, umax_dry))
nominal_policy_jnp = lambda x, t: jnp.atleast_1d(jnp.clip(-feedback_gain * (x[..., 1] - desired_vel), umin_dry, umax_dry))

### Setting up the safety filter

In [52]:
alpha = lambda x: 0.2 * x
alpha_ice = lambda x: 5. * x

acc_asif_dry = ControlAffineASIF(acc_dry, acc_cbf_dry, alpha=alpha, nominal_policy=nominal_policy, umin=umin_dry, umax=umax_dry)
acc_asif_ice = ControlAffineASIF(acc_ice, acc_cbf_ice, alpha=alpha_ice, nominal_policy=nominal_policy, umin=umin_ice, umax=umax_ice)
acc_asif_global = ControlAffineASIF(acc_ice, acc_cbf_ice, alpha=alpha_ice, nominal_policy=nominal_policy, umin=umin_dry, umax=umax_dry)
acc_asif_dry_optimal = ControlAffineASIF(acc_dry, refined_cbf_dry, alpha=alpha, nominal_policy=nominal_policy, umin=umin_dry, umax=umax_dry)
acc_asif_dry_refined = ControlAffineASIF(acc_dry, refined_cbf_dry, alpha=alpha, nominal_policy=nominal_policy, umin=umin_dry, umax=umax_dry)

Visualizations for paper + demo + videos

Safe set visualization

In [54]:
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size': '28',
    'text.usetex': False,
    'pgf.rcfonts': False,
})

colors = sns.color_palette("tab10")
chosen_colors = [(0.5, 0.5, 0.5)]
chosen_colors.append(colors[0])
chosen_colors.append(colors[1])
# chosen_colors.append((0.1, 0.1, 0.1))
# chosen_colors.append((0.7, 0.7, 0.7))

In [55]:
colors = [(0.3, 0.3, 0.3)]
colors += [(sns.color_palette("RdYlGn_r", 7)[0])]
colors += [(sns.color_palette("RdYlGn_r", 9)[6])]
colors += [(sns.color_palette("RdYlGn_r", 9)[8])]
colors += [(4 / 255, 101 / 255, 4 / 255)]
colors = np.array(colors)

alt_colors = sns.color_palette("pastel", 9).as_hex()

In [56]:
toggle_hjr_visualization = False
save_video = True
video_file_name = "acc.gif"

### Online: Safety filter visualization

In [69]:
experiment_paper = RolloutTrajectory('acc_example', start_x=x0, n_sims_per_start=n_sims_per_start, t_sim=t_sim)

In [ ]:
method_index = 2 # 0 is vanilla, 1 is ours, 2 is global CBF

if method_index == 0:
    phase1_controller = acc_asif_dry
elif method_index == 2:
    phase1_controller = acc_asif_global
else:
    phase1_controller = acc_asif_dry_refined

paper_results, jump_state, traj_all = experiment_paper.run_hybrid(acc_dry, acc_ice, {'nominal': nominal_policy, 'Analytical': phase1_controller}, {'nominal': nominal_policy, 'Analytical': acc_asif_ice}, switch_position)

if method_index == 0:
    traj_all_bad = traj_all
elif method_index == 2:
    traj_all_global = traj_all
else:
    traj_all_good = traj_all

In [71]:
save_figure = False
figure_file_name = "acc11.png"
index_method_dict = {
    0: "vanilla",
    1: "ours",
    2: "global"
}

jump state:  [[100.12027551  20.22781669  45.59972449]]


In [59]:
video_file_name = "baseline-bad.gif"

In [80]:
# save trajectories data
with open(dataset_path + "/traj_vanilla.npy", "wb") as f:
   jnp.save(f, traj_all_bad)
with open(dataset_path + "/traj_ours.npy", "wb") as f:
   jnp.save(f, traj_all_good)
with open(dataset_path + "/traj_global.npy", "wb") as f:
   jnp.save(f, traj_all_global)

In [92]:
with open(dataset_path + "/traj_vanilla.npy", "rb") as f:
    traj_all_bad = jnp.load(f)
with open(dataset_path + "/traj_ours.npy", "rb") as f:
    traj_all_good = jnp.load(f)
with open(dataset_path + "/traj_global.npy", "rb") as f:
    traj_all_global= jnp.load(f)

In [87]:
method_index = 0 # 0 is vanilla, 1 is ours, 2 is global CBF

if method_index == 0:
    traj_all = traj_all_bad
elif method_index == 2:
    traj_all = traj_all_global
else:
    traj_all = traj_all_good

In [94]:
colors = sns.color_palette("tab10")
alt_colors = sns.color_palette("pastel", 9).as_hex()
toggle_hjr_visualization = False
save_video = True
results_path = "../paper_results/"
video_file_name = results_path + f"acc_{index_method_dict[method_index]}.gif"

In [ ]:
params = {'axes.labelsize': 28,'axes.titlesize':28, 'font.size': 28, 'legend.fontsize': 28, 
          'xtick.labelsize': 28, 'ytick.labelsize': 28, 'lines.linewidth': 5}
matplotlib.rcParams.update(params)

def render_frame(i, colorbar=False):
    global cont
    # global cont_new
    for c in cont.collections:
        c.remove()
    # timestamp.set_text("Time step = {}".format(i))

    if traj_all[i*20][0] >= switch_position - 2. and traj_all[i*20][0] <= switch_position + 2.:
        switch_index = i
        newpoint, = ax.plot(traj_all[i*20][1], traj_all[i*20][2], color="red", marker="o")
        ## for good traj
        if method_index == 1:
            ax.text(14, 50, 'switching \n state', fontsize = 22, color = 'red')
            ax.arrow(17,48, 1.6,-1.5,width=0.2, color = 'red')
        # for bad traj
        elif method_index == 0:
            ax.text(15, 49, 'switching \n state', fontsize = 22, color = 'red')
            ax.arrow(19.6,47, 1.6,-1.5,width=0.2, color = 'red')
        # for global CBF
        else:
            ax.text(12, 53, 'switching \n state', fontsize = 22, color = 'red')
            ax.arrow(16,52, 1.6,-1.5,width=0.2, color = 'red')

    else:
        newpoint, = ax.plot(traj_all[i*20][1], traj_all[i*20][2], color='blue', marker="o")
    
    if traj_all[i*20][0] >= switch_position + 0.5:
        cont = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_ice[-1][i].T, levels=[0, 200], 
                 colors='green', alpha=0.3)
    else:
        cont = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_refine_dry[-1][i].T, levels=[0, 200], 
                 colors='green', alpha=0.3)
    
    # cont = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_hybrid[-1][i].T, levels=[0, 200], 
    #              colors='green', alpha=0.3)


    # if traj_all[i*20][0] >= switch_position - 2. and traj_all[i*20][0] <= switch_position + 2.:
    #     newpoint, = ax.plot(traj_all[i*20][1], traj_all[i*20][2], color="red", marker="o")
    #     print(traj_all[i*20][0])
    # else:
    #     newpoint, = ax.plot(traj_all[i*20][1], traj_all[i*20][2], color="blue", marker="o")
    # cont = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_hybrid[-1][i].T, levels=[0, 200], 
    #              colors='green', alpha=0.3)
    
    # if toggle_hjr_visualization:
    #     global cont2
    #     for c in cont2.collections:
    #         c.remove()
    #     cont2 = ax.contour(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_hjr_dry[i][0].T, levels=[0],
    #                        colors=np.array([chosen_colors[0]]))
    

proxy = [] 

fig, ax = plt.subplots(figsize=(9, 16))

obstacle_viz = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], obstacle_dry[0].T, levels=[-100, 0], 
                 colors='red', alpha=0.3)


cs = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_dry[-1][0].T, levels=[0, 200], 
                 colors='grey', alpha=0.3)

cs = ax.contourf(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_ice[-1][0].T, levels=[0, 200], 
                 colors='black', alpha=0.3)

cont = ax.contour(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_dry[0][0].T, levels=[0], 
                  colors=np.array([colors[3]]))   # Initial CBF

proxy += [plt.Rectangle((0,0),1,1,ec =colors[3], fc='white', lw=5)
        for pc in cont.collections]

proxy += [plt.Rectangle((0,0),1,1,ec =colors[-1], fc='white', lw=5) for pc in cont.collections]

timestamp = ax.text(0.05, 0.9, "", transform=ax.transAxes)

# cont = ax.contour(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_dry[0][0].T, levels=[0], 
#                   colors=np.array([colors[-1]]))

# cont = ax.contour(grid.coordinate_vectors[1], grid.coordinate_vectors[2], target_values_ice[-1][0].T, levels=[0], 
#                   colors='black')

ax.grid()
proxy2 = []
proxy2 += [plt.Rectangle((0,0),1,1,fc = alt_colors[3], ec=alt_colors[3], alpha=0.3)
        for pc in cs.collections]

proxy2 += [plt.Rectangle((0,0),1,1,fc = 'grey', ec='grey', alpha=0.3)
        for pc in cs.collections]

proxy2 += [plt.Rectangle((0,0),1,1, fc='darkgreen', ec='darkgreen', alpha=0.3) for pc in obstacle_viz.collections]

proxy2 += [plt.Rectangle((0,0),1,1, fc='green', ec='green', alpha=0.3) for pc in obstacle_viz.collections]

legend_entries = ["$\partial \mathcal{C}_h$", "$\partial \mathcal{C}_h(t)$"]
if toggle_hjr_visualization:
    legend_entries += ["$\partial \mathcal{C}_{\ell}(t)$"]
# leg = ax.legend(proxy, legend_entries,
#           loc='center', bbox_to_anchor=(0.5, 0.2), ncol=3, columnspacing=1.5, handletextpad=0.4,
#           facecolor=[0.8, 0.8, 0.8], edgecolor='black')

ax.arrow(29,92,-2,-8,width=0.2, color = 'blue')
ax.text(27, 94, 'trajectory', fontsize = 22, color = 'blue')

ax.legend(proxy2, ["unsafe area", "safe set $\mathcal{C}_{dry}$", "safe set $\mathcal{C}_{ice}$", "$\mathcal{C}_{dry, ice}$"],
          loc='center', bbox_to_anchor=(0.72, 0.2), ncol=1, columnspacing=1.5, handletextpad=0.4,
          facecolor=[0.8, 0.8, 0.8], edgecolor='black')

ax.set_ylabel("Distance between vehicles ($z$) [m]")
ax.set_xlabel("Ego velocity ($v$) [m/s]")

render_frame(0, False)
animation = anim.FuncAnimation(fig, render_frame, 50, interval=200)
save_video = True
if save_video:
   animation.save(video_file_name, writer='ffmpeg')